In [1]:
import pandas as pd
import numpy as np
import datetime
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Information about the data
The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics tracks the on-time performance of domestic flights operated by large air carriers. Summary information on the number of on-time, delayed, canceled, and diverted flights is published in DOT's monthly Air Travel Consumer Report and in this dataset of 2015 flight delays and cancellations.

#### Acknowledgements
The flight delay and cancellation data was collected and published by the DOT's Bureau of Transportation Statistics.

In [2]:
#ingest data
#airlines= pd.read_csv('airlines.csv')
#airports= pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

pd.set_option("display.max_columns", 100)

### Data Processing

#### Cleaning the data and augmenting


###### For Flights
- convert to columns to a single date 
- convert integer columns to a time or minutes from data
- add if its a holiday, weekend as a binary term

In [3]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Take all the date columns and combine into a single column
#Currently there is a separate column for month, day, year
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

#remove redundant columns
flights.drop(['YEAR','MONTH', 'DAY'], axis=1,inplace=True)

In [5]:
df = pd.DataFrame({"column1": ["194", "123", "194"]})
df["column1"].replace({194: "x", 123: "y"}, inplace=True)
print(df)

  column1
0     194
1     123
2     194


In [6]:
#################################################################
#Realized that there are a bunch of 5 digit integers as the origin_airport instead of the 3 digit code
#display(successful_flights['ORIGIN_AIRPORT'].unique())
#display(successful_flights['DESTINATION_AIRPORT'].unique())
#################################################################


unique_codes=list(flights['ORIGIN_AIRPORT'].unique())
unique_codes_arr = []
#we want only the number codes in the data
for x in unique_codes:
    if isinstance(x,int):
        unique_codes_arr.append(str(x))
    elif x.isnumeric():
        unique_codes_arr.append(x)
        
unique_codes=list(flights['DESTINATION_AIRPORT'].unique())
unique_codes_dest = []
#we want only the number codes in the data
for x in unique_codes:
    if isinstance(x,int):
        unique_codes_dest.append(str(x))
    elif x.isnumeric():
        unique_codes_dest.append(x)

#build our dictionary to translate the codes to the letter codes        
airport_numeric_codes = pd.read_csv('airport_num_codes.csv')
airport_numeric_codes['Description']= airport_numeric_codes['Description'].astype(str)


airport_letter_codes = pd.read_csv('airports_letter_codes.csv')
airport_letter_codes['Description']= airport_letter_codes['Description'].astype(str)

airport_codes = airport_numeric_codes.merge(airport_letter_codes, on=['Description'])
airport_codes.drop(['Description'], axis=1, inplace=True)
    
code_dict = dict(zip(airport_codes.Codes,airport_codes.Code))

#since this is a huge dictionary, need to trim it (exponentially improves the speed)
#there is an integer version and string version present in the data
new_data_arr_int = {str(k): v for k, v in code_dict.items() if str(k) in unique_codes_arr}
new_data_arr_str = {k: v for k, v in code_dict.items() if str(k) in unique_codes_arr}

new_data_dest_int = {str(k): v for k, v in code_dict.items() if str(k) in unique_codes_dest}
new_data_dest_str = {k: v for k, v in code_dict.items() if str(k) in unique_codes_dest}

#replace the number codes with letter ones
flights['ORIGIN_AIRPORT'].replace(new_data_arr_int,inplace=True)
flights['ORIGIN_AIRPORT'].replace(new_data_arr_str,inplace=True)
print(flights['ORIGIN_AIRPORT'].unique())
flights['DESTINATION_AIRPORT'].replace(new_data_dest_int,inplace=True)
flights['ORIGIN_AIRPORT'].replace(new_data_dest_str,inplace=True)
print(flights['DESTINATION_AIRPORT'].unique())



['ANC' 'LAX' 'SFO' 'SEA' 'LAS' 'DEN' 'SLC' 'PDX' 'FAI' 'MSP' 'PHX' 'SJU'
 'PBG' 'IAG' 'PSE' 'BQN' 'ORD' 'GEG' 'HNL' 'ONT' 'MCO' 'BOS' 'HIB' 'ABR'
 'MAF' 'DFW' 'MKE' 'IAH' 'BNA' 'BRO' 'VPS' 'BOI' 'BJI' 'SGF' 'PHL' 'SBN'
 'RDD' 'EUG' 'IAD' 'BUF' 'PWM' 'JFK' 'CRP' 'PIA' 'FAT' 'SMF' 'AUS' 'MCI'
 'ATL' 'JAX' 'MFR' 'IDA' 'MSN' 'DCA' 'SAT' 'CHS' 'SBA' 'SMX' 'IND' 'CLE'
 'GSP' 'BDL' 'ABI' 'RIC' 'BFL' 'OMA' 'RDM' 'FLL' 'CID' 'TPA' 'SYR' 'ROC'
 'TYR' 'LAN' 'XNA' 'GSO' 'EWR' 'PBI' 'RSW' 'OAK' 'PVD' 'RNO' 'PIT' 'ABQ'
 'MIA' 'BWI' 'LGA' 'TUL' 'LIT' 'MSY' 'OKC' 'ATW' 'PNS' 'MEM' 'TYS' 'MHT'
 'SAV' 'CLT' 'GRB' 'ABE' 'JAN' 'OAJ' 'FAR' 'ERI' 'LEX' 'CWA' 'MSO' 'TTN'
 'AMA' 'CLL' 'HOU' 'JLN' 'MLI' 'RDU' 'CVG' 'MHK' 'MOB' 'TLH' 'BHM' 'CAE'
 'TXK' 'ACY' 'DTW' 'RAP' 'TUS' 'EAU' 'DLH' 'FSD' 'INL' 'CMX' 'SPI' 'CLD'
 'COD' 'CMH' 'LRD' 'PSC' 'CPR' 'ACV' 'DAL' 'PAH' 'MRY' 'ESC' 'ISN' 'PSP'
 'MFE' 'STL' 'BTV' 'FSM' 'AEX' 'SPS' 'ACT' 'SJT' 'MTJ' 'GCC' 'OGG' 'SJC'
 'GUC' 'ORF' 'MOT' 'MLU' 'KOA' 'SAN' 'LAW' 'PIB' 'M

In [7]:
#American holidays for 2015
american_holidays = ['2015-01-01', '2015-01-19','2015-02-16','2015-05-25','2015-07-03','2015-09-07','2015-10-12','2015-11-11','2015-11-26','2015-12-25']

# - Change time from integer to actual time/date or minutes from a time
def format_time(value):
    if np.isnan(value):
        return np.nan
    if value == 2400: value = 0
    value = "{0:04d}".format(int(value))
    formatted_time = datetime.time(int(value[0:2]), int(value[2:4]))
    return formatted_time

#Add a binary term if it is a holiday or weekend
flights['weekend'] = np.where(flights['DAY_OF_WEEK']> 5, True, False)
flights['holiday'] = np.where(flights['DATE'].isin(american_holidays), True, False)

#Add a binary term if it is a long flight > 1400 miles
flights['long_flight'] = np.where(flights['DISTANCE']>1400, True, False)


#########################################################
# subset of data where flight is either cancelled or diverted
#x = data[(data['Cancelled']==1) 
########################################################
successful_flights = flights[np.isnan(flights['ARRIVAL_TIME'])==False]
cancelled_flights = flights[np.isnan(flights['ARRIVAL_TIME'])]

#Cancelled flights, 
# - delete all empty columns
cancelled_flights.drop(['DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','ELAPSED_TIME','AIR_TIME',
                        'TAXI_IN','WHEELS_ON','ARRIVAL_TIME','ARRIVAL_DELAY','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY'
                        ,'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'], axis=1,inplace=True)
# - change time format
cancelled_flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_time)
cancelled_flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_time)

#Successful flights,
# - fill NaN values with 0 for AIR_SYSTEM_DELAY, SECURITY_DELAY , AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY
successful_flights=successful_flights.fillna(0)
# - delete diverted, cancelled and cancelled reason
successful_flights.drop(['DIVERTED','CANCELLED', 'CANCELLATION_REASON'], axis=1,inplace=True)

# - change time format
successful_flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_time)
successful_flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_time)
successful_flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_time)
successful_flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_time)
successful_flights['WHEELS_OFF'] = flights['WHEELS_OFF'].apply(format_time)
successful_flights['WHEELS_ON'] = flights['WHEELS_ON'].apply(format_time)

In [8]:
#make sure everything looks right
successful_flights.head(5)

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,weekend,holiday,long_flight
0,4,AS,98,N407AS,ANC,SEA,00:05:00,23:54:00,-11.0,21.0,00:15:00,205.0,194.0,169.0,1448,04:04:00,4.0,04:30:00,04:08:00,-22.0,0.0,0.0,0.0,0.0,0.0,2015-01-01,False,True,True
1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,00:02:00,-8.0,12.0,00:14:00,280.0,279.0,263.0,2330,07:37:00,4.0,07:50:00,07:41:00,-9.0,0.0,0.0,0.0,0.0,0.0,2015-01-01,False,True,True
2,4,US,840,N171US,SFO,CLT,00:20:00,00:18:00,-2.0,16.0,00:34:00,286.0,293.0,266.0,2296,08:00:00,11.0,08:06:00,08:11:00,5.0,0.0,0.0,0.0,0.0,0.0,2015-01-01,False,True,True
3,4,AA,258,N3HYAA,LAX,MIA,00:20:00,00:15:00,-5.0,15.0,00:30:00,285.0,281.0,258.0,2342,07:48:00,8.0,08:05:00,07:56:00,-9.0,0.0,0.0,0.0,0.0,0.0,2015-01-01,False,True,True
4,4,AS,135,N527AS,SEA,ANC,00:25:00,00:24:00,-1.0,11.0,00:35:00,235.0,215.0,199.0,1448,02:54:00,5.0,03:20:00,02:59:00,-21.0,0.0,0.0,0.0,0.0,0.0,2015-01-01,False,True,True


###### For Airport:
- get average delay for each source airport 
- average taxi time for each source airport
- most common delay reason

In [9]:
#Building the new Airports dataframe
airport_data = successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'ARRIVAL_DELAY':'mean'})
airport_data['destination_airport_avg_delay'] = successful_flights.groupby(by='DESTINATION_AIRPORT').agg({'ARRIVAL_DELAY':'mean'})
airport_data['airport_avg_taxi_out_time'] = successful_flights.groupby(by='DESTINATION_AIRPORT').agg({'TAXI_OUT':'mean'})
airport_data['airport_avg_taxi_in_time'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'TAXI_IN':'mean'})
airport_data['total_airline_delay'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'AIRLINE_DELAY':'sum'})
airport_data['total_security_delay'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'SECURITY_DELAY':'sum'})
airport_data['total_air_system_delay'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'AIR_SYSTEM_DELAY':'sum'})
airport_data['total_late_aircraft_delay'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'LATE_AIRCRAFT_DELAY':'sum'})
airport_data['total_weather_delay'] =  successful_flights.groupby(by='ORIGIN_AIRPORT').agg({'WEATHER_DELAY':'sum'})
airport_data.rename(columns={'ARRIVAL_DELAY': 'source_airport_avg_delay'},inplace=True)
airport_data

,source_airport_avg_delay,destination_airport_avg_delay,airport_avg_taxi_out_time,airport_avg_taxi_in_time,total_airline_delay,total_security_delay,total_air_system_delay,total_late_aircraft_delay,total_weather_delay
ORIGIN_AIRPORT,,,,,,,,,
ABE,6.262434,5.700671,18.836242,8.397493,9875.0,0.0,7894.0,12238.0,355.0
ABI,2.533800,4.188114,15.113494,10.640973,9189.0,46.0,4404.0,7518.0,5205.0
ABQ,4.403061,5.610846,14.721771,7.661051,62969.0,282.0,38227.0,100148.0,9496.0
ABR,9.027285,-3.381381,18.288288,7.755798,4877.0,30.0,1589.0,2979.0,376.0
ABY,7.867925,8.530355,17.924399,7.855346,5723.0,0.0,1328.0,4744.0,0.0
...,...,...,...,...,...,...,...,...,...
WRG,4.879774,5.260671,11.254573,4.161245,646.0,30.0,594.0,6054.0,112.0
WYS,-1.865385,4.173077,20.192308,5.975962,258.0,0.0,98.0,472.0,22.0
XNA,10.256452,8.694285,19.413283,11.420464,40216.0,11.0,44067.0,60494.0,12256.0


###### For Airline
- average delay for each carrier
- average difference between schedule and elapsed time
- percentage of flights delayed by carrier 
- average wheels up 
- most common delay reason

In [10]:
airline_avg_arr_delay = successful_flights.groupby(by='AIRLINE').agg({'ARRIVAL_DELAY':'mean'})
airline_avg_dep_delay = successful_flights.groupby(by='AIRLINE').agg({'DEPARTURE_DELAY':'mean'})
sched_elapsed_diff = successful_flights['SCHEDULED_TIME']- successful_flights['ELAPSED_TIME']
successful_flights["sched_elapsed_diff"] = sched_elapsed_diff
airline_scheduled_elapsed_diff = successful_flights.groupby(by='AIRLINE').agg({'sched_elapsed_diff':'mean'})
successful_flights['DELAYED'] = np.where(successful_flights['ARRIVAL_DELAY']>10, 1, 0)
num_delayed_flight = successful_flights.groupby(by='AIRLINE').agg({'DELAYED':'sum'}) 
total_num_flight = successful_flights.groupby(by='AIRLINE').agg({'SCHEDULED_DEPARTURE':'count'})  

air_delay = successful_flights.groupby(by='AIRLINE').agg({'AIR_SYSTEM_DELAY':'sum'})
weather_delay = successful_flights.groupby(by='AIRLINE').agg({'WEATHER_DELAY':'sum'})
security_delay = successful_flights.groupby(by='AIRLINE').agg({'SECURITY_DELAY':'sum'}) 
airline_delay = successful_flights.groupby(by='AIRLINE').agg({'AIRLINE_DELAY':'sum'})
aircraft_delay = successful_flights.groupby(by='AIRLINE').agg({'LATE_AIRCRAFT_DELAY':'sum'})
#percent_delayed_flight.columns=['AIRLINE','percent_delayed_flight']
#print(percent_delayed_flight.head(5))
delay_reason_totals = air_delay.join(weather_delay, on='AIRLINE') 
delay_reason_totals = delay_reason_totals.join(security_delay, on='AIRLINE')
delay_reason_totals = delay_reason_totals.join(airline_delay, on='AIRLINE')
delay_reason_totals = delay_reason_totals.join(aircraft_delay, on='AIRLINE')

airlines= delay_reason_totals.join(airline_avg_arr_delay, on='AIRLINE')
airlines= airlines.join(airline_avg_dep_delay,on='AIRLINE')
airlines= airlines.join(airline_scheduled_elapsed_diff,on='AIRLINE')

airlines['percent_delayed_flight']=num_delayed_flight["DELAYED"]/total_num_flight["SCHEDULED_DEPARTURE"]
#airlines= airlines.join(percent_delayed_flight,on='AIRLINE')
airlines

,AIR_SYSTEM_DELAY,WEATHER_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,ARRIVAL_DELAY,DEPARTURE_DELAY,sched_elapsed_diff,percent_delayed_flight
AIRLINE,,,,,,,,,
AA,1760561.0,467420.0,16158.0,2753994.0,2833302.0,3.442102,8.848322,5.880078,0.215588
AS,301478.0,38832.0,5825.0,347425.0,381417.0,-0.975124,1.730191,3.041505,0.164102
B6,991461.0,115770.0,11417.0,1074056.0,1417496.0,6.661464,11.481192,5.361057,0.258616
DL,1779383.0,602901.0,3910.0,2707569.0,2136128.0,0.186396,7.345122,7.429001,0.162727
EV,1687894.0,169313.0,0.0,2363973.0,2628976.0,6.565578,8.642840,2.375179,0.231596
F9,581234.0,21616.0,0.0,346950.0,634039.0,12.485166,13.305449,1.064202,0.300772
HA,6241.0,11429.0,401.0,196422.0,126699.0,2.021710,0.475208,-1.399380,0.155271
MQ,930774.0,402305.0,7049.0,1055033.0,1417073.0,6.443083,9.995351,3.763251,0.247542
NK,941423.0,44088.0,5147.0,471115.0,701218.0,14.452979,15.899604,1.635149,0.338373


###### New Daily Data (each day is an entry):
- average delay on all flights
- number of flights 
- number of planes flying each day
- number of unique flight plans

In [11]:
#building daily data df
average_arr_delay_daily = successful_flights.groupby(by='DATE').agg({'ARRIVAL_DELAY':'mean'})
average_dep_delay_daily = successful_flights.groupby(by='DATE').agg({'DEPARTURE_DELAY':'mean'})
num_flight_daily = successful_flights.groupby(by='DATE').agg({'SCHEDULED_DEPARTURE':'count'})
daily_data = average_arr_delay_daily.join(average_dep_delay_daily,on='DATE')
daily_data = daily_data.join(num_flight_daily,on='DATE')
daily_data.rename(columns = {'ARRIVAL_DELAY':'average_arr_delay_daily','DEPARTURE_DELAY':'average_dep_delay_daily','SCHEDULED_DEPARTURE':'num_flight_daily'}, inplace = True) 
daily_data


,average_arr_delay_daily,average_dep_delay_daily,num_flight_daily
DATE,,,
2015-01-01,5.346142,9.573887,13480
2015-01-02,9.825171,12.635415,16479
2015-01-03,25.394367,25.127966,15090
2015-01-04,31.920762,31.519417,15914
2015-01-05,18.780932,20.881491,16100
...,...,...,...
2015-12-27,27.976374,30.801919,15322
2015-12-28,24.406002,28.815338,14096
2015-12-29,26.255322,32.235970,15502


###### For Predictions

- keep the cancelled and successful flight data together
- remove the useless data
- add average airline delay
- add average source airport delay
- add total number of flights that day from the source
- Prediction task one: predicts the delay in minutes for each flight
- Prediction task two: predicts if it is delayed more than 10mins or not

In [12]:
#Remove the useless data
flight_predictions = flights.drop(['FLIGHT_NUMBER','TAIL_NUMBER'], axis=1)

flight_predictions['airline_avg_arrival_delay'] = flight_predictions.groupby('AIRLINE')['ARRIVAL_DELAY'].transform(np.mean)
flight_predictions['airline_avg_departure_delay'] = flight_predictions.groupby('AIRLINE')['DEPARTURE_DELAY'].transform(np.mean)

##gather aggregrate data for airports
#get mean arrival delay for each airport, rename the column
flight_predictions['source_airport_avg_departure_delay'] = flight_predictions.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].transform(np.mean)

#get mean departure delay for each airport, rename the column
flight_predictions['destination_airport_avg_delay'] = flight_predictions.groupby('DESTINATION_AIRPORT')['ARRIVAL_DELAY'].transform(np.mean)

#get total flights by source, rename the column
flight_predictions['total_flights_source'] = flight_predictions.groupby('ORIGIN_AIRPORT')['SCHEDULED_DEPARTURE'].transform(np.size)

#get total flights by destination, rename the column
flight_predictions['total_flights_destination'] = flight_predictions.groupby('DESTINATION_AIRPORT')['SCHEDULED_DEPARTURE'].transform(np.size)

flight_predictions

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,weekend,holiday,long_flight,airline_avg_arrival_delay,airline_avg_departure_delay,source_airport_avg_departure_delay,destination_airport_avg_delay,total_flights_source,total_flights_destination
0,4,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,False,True,True,-0.976563,1.785801,3.249737,1.991544,17239,111527
1,4,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,False,True,True,3.451372,8.900856,10.216556,6.808427,212401,22678
2,4,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,False,True,True,3.706209,6.141137,10.760883,2.029852,162178,100851
3,4,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,False,True,True,3.451372,8.900856,10.216556,5.925841,212401,69696
4,4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,False,True,True,-0.976563,1.785801,6.538400,1.779463,121287,16075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,4,B6,LAX,BOS,2359,2355.0,-4.0,22.0,17.0,320.0,298.0,272.0,2611,749.0,4.0,819,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,False,False,True,6.677861,11.514353,10.216556,6.192347,212401,108409
5819075,4,B6,JFK,PSE,2359,2355.0,-4.0,17.0,12.0,227.0,215.0,195.0,1617,427.0,3.0,446,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,False,False,True,6.677861,11.514353,11.755940,9.951417,102114,755
5819076,4,B6,JFK,SJU,2359,2350.0,-9.0,17.0,7.0,221.0,222.0,197.0,1598,424.0,8.0,440,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,False,False,True,6.677861,11.514353,11.755940,6.883341,102114,24783
5819077,4,B6,MCO,SJU,2359,2353.0,-6.0,10.0,3.0,161.0,157.0,144.0,1189,327.0,3.0,340,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,False,False,False,6.677861,11.514353,11.508909,6.883341,120029,24783


#### Put data into new csv files for Streamlit dashboard

- For flights -> flights_clean.csv
- For cancelled_flights -> cancelled_flights_clean.csv
- For airports -> airports_clean.csv - done
- For airlines -> airlines_clean.csv
- For daily info -> daily_clean.csv

In [14]:
successful_flights.to_csv('flights_clean.csv', index = False, header=True)
cancelled_flights.to_csv('cancelled_flights_clean.csv', index = False, header=True)
airport_data.to_csv('airports_clean.csv', index = True, header=True)
airlines.to_csv('airlines_clean.csv', index = True, header=True)
daily_data.to_csv('daily_clean.csv', index = True, header=True)
flight_predictions.to_csv('flight_predictions.csv', index = False, header=True)